# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Low Rank Adaptation (LoRA)
* Model: GPT-2
* Evaluation approach:  Hugging Face Trainer with accuracy as the evaluation metric
* Fine-tuning dataset: AG News

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [74]:
# Load the dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2Config, GPT2ForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, EvalPrediction

# Load the AG News dataset
dataset = load_dataset("ag_news", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"] = dataset["train"].select(range(0, int(0.01 * len(dataset['train']))))
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 960
})

In [75]:
dataset["test"] = dataset["test"].select(range(0, int(0.1 * len(dataset['train']))))
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 96
})

In [76]:
dataset["train"][0]

{'text': 'Motorola to Invest  #36;90 Million in China R D Center (Reuters) Reuters - Motorola Inc. said on Friday it plans\\to invest  #36;90 million in a new research facility in Beijing as\\the U.S. mobile phone maker fights to wrest back top position\\in the domestic handset market.',
 'label': 3}

In [77]:
# Preprocess the dataset

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", num_labels=4)

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

tokenized_dataset["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 960
})

In [78]:
# Set up the model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=4,
    id2label={0: "world", 1: "sports", 2: "business", 3: "sci/tech"},
    label2id={"world": 0, "sports": 1, "business": 2, "sci/tech": 3},
)

print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [79]:
# Train the model

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results/ag_news",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.852318,0.843750
2,No log,0.505229,0.875000
3,No log,0.410959,0.864583
4,No log,0.404622,0.885417
5,No log,0.356657,0.895833
6,No log,0.380001,0.875000
7,No log,0.403667,0.875000
8,No log,0.415645,0.875000
9,0.379000,0.420695,0.875000
10,0.379000,0.425439,0.875000


Checkpoint destination directory ./results/ag_news/checkpoint-60 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-120 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-180 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-240 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-300 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-360 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news/checkpoint-420 already exists and is n

TrainOutput(global_step=600, training_loss=0.3351627318064372, metrics={'train_runtime': 138.7535, 'train_samples_per_second': 69.187, 'train_steps_per_second': 4.324, 'total_flos': 235779680868480.0, 'train_loss': 0.3351627318064372, 'epoch': 10.0})

In [80]:
# Show the performance of the model on the test set
trainer.evaluate()

# Save a Trained Model
model.save_pretrained('model/base_model')

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [81]:
from peft import LoraConfig, get_peft_model, TaskType

# Create a PEFT Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS,
)

# training and testing sets
train_lora = tokenized_dataset['train'].rename_column('label', 'labels')
test_lora = tokenized_dataset['test'].rename_column('label', 'labels')

In [82]:
# Create a PEFT Model
peft_model = get_peft_model(model, lora_config)

In [83]:
# Train the PEFT model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

peft_trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./results/ag_news_peft",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.359753,0.895833
2,No log,0.365546,0.895833
3,No log,0.369744,0.895833
4,No log,0.371803,0.895833
5,No log,0.379368,0.885417
6,No log,0.379186,0.885417
7,No log,0.380225,0.875000
8,No log,0.381692,0.875000
9,0.168200,0.381656,0.875000
10,0.168200,0.382326,0.875000


Checkpoint destination directory ./results/ag_news_peft/checkpoint-60 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/checkpoint-120 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/checkpoint-180 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/checkpoint-240 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/checkpoint-300 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/checkpoint-360 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/ag_news_peft/che

TrainOutput(global_step=600, training_loss=0.16403687318166096, metrics={'train_runtime': 63.6357, 'train_samples_per_second': 150.859, 'train_steps_per_second': 9.429, 'total_flos': 240638151962880.0, 'train_loss': 0.16403687318166096, 'epoch': 10.0})

In [84]:
# Final evaluation after fine-tuning
eval_results = peft_trainer.evaluate()
print(f"Final evaluation results after fine-tuning: {eval_results}")

# Save a Trained PEFT Model
peft_model.save_pretrained('model/peft_model')

Final evaluation results after fine-tuning: {'eval_loss': 0.3597525358200073, 'eval_accuracy': 0.8958333333333334, 'eval_runtime': 0.3424, 'eval_samples_per_second': 280.366, 'eval_steps_per_second': 17.523, 'epoch': 10.0}


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [85]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained("model/base_model", num_labels=4)
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [86]:
# Load the PFET model
peft_model = AutoModelForSequenceClassification.from_pretrained("model/peft_model", num_labels=4)
print(peft_model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=768, bias=False)
              )

In [91]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
      [0, 1, 3, 9, 10, 15, 18,  22, 30, 31, 38, 43, 45, 50, 61, 70, 72, 80, 84, 89, 91]
)

results = trainer.predict(items_for_manual_review)

df = pd.DataFrame(
    {
        "sms": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,"Foulke, Wakefield are happy to pitch in As much as the Red Sox relished thumping the Yankees, they took special satisfaction from the outings of two key pitchers who recently have struggled, Keith Foulke and Tim Wakefield .",1,1
1,"Nfld. oil spill more serious than originally thought ST. JOHN #39;S, Nfld. -- An oil spill on Newfoundland #39;s Grand Banks is larger than first reported. The Canada-Newfoundland Offshore Petroleum Board now says that up to 1,000 barrels of oil may have spilled into",2,2
2,"Death toll in Asian quake disaster passes 144,000 (AFP) AFP - The number of people killed in the massive earthquake and tidal waves that hit Indian Ocean shorelines a week ago passed 144,000.",0,0
3,Lampard strikes as England cruise past Wales MANCHESTER: A stunning goal by captain David Beckham guided England to a convincing 2-0 victory over Wales in their World Cup qualifier at his former Old Trafford home yesterday.,1,1
4,"Billy Reay, Hawks #39; winningest coach, dead at 86 Billy Reay, the winningest coach in Blackhawks history and a two-time Stanley Cup champion as a player, died Thursday at age 86. Reay, who had suffered from cancer",1,1
5,"Pandas benefit from wireless net The world #39;s dwindling panda population is getting a helping hand from a wireless internet network. The Wolong Nature Reserve in the Sichuan Province of southwest China is home to 20 of the remaining 1,500 giant pandas in the world.",3,3
6,"Stocks Up in Light Volume Ahead of Convention NEW YORK (Reuters) - U.S. stocks gained on Friday after reports showed U.S. economic growth in the second quarter matched economists' forecasts and consumer sentiment picked up above expectations in late August, but volume was feather light ahead of the upcoming Republican convention in New York City.",2,2
7,"Mutual fund votes show limits to change When corporate America came looking for raises last year, Boston #39;s largest mutual funds routinely turned them down, voting in hundreds of cases against lucrative",2,2
8,Rams Offensive Line Strong in Debut (AP) AP - A makeshift offensive line came together at just the right time for the St. Louis Rams.,1,1
9,"Rival Targets Apple's ITunes Customers (AP) AP - For more than a year, Apple Computers Inc. enjoyed singular success selling songs exclusively to users of its iPod portable music player. Now, it's got rival RealNetworks Inc. trying to lure iTunes customers away.",3,3


In [90]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
   [0, 1, 3, 9, 10, 15, 18,  22, 30, 31, 38, 43, 45, 50, 61, 70, 72, 80, 84, 89, 91]
)

results = peft_trainer.predict(items_for_manual_review)

df = pd.DataFrame(
    {
        "sms": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,"Foulke, Wakefield are happy to pitch in As much as the Red Sox relished thumping the Yankees, they took special satisfaction from the outings of two key pitchers who recently have struggled, Keith Foulke and Tim Wakefield .",1,1
1,"Nfld. oil spill more serious than originally thought ST. JOHN #39;S, Nfld. -- An oil spill on Newfoundland #39;s Grand Banks is larger than first reported. The Canada-Newfoundland Offshore Petroleum Board now says that up to 1,000 barrels of oil may have spilled into",2,2
2,"Death toll in Asian quake disaster passes 144,000 (AFP) AFP - The number of people killed in the massive earthquake and tidal waves that hit Indian Ocean shorelines a week ago passed 144,000.",0,0
3,Lampard strikes as England cruise past Wales MANCHESTER: A stunning goal by captain David Beckham guided England to a convincing 2-0 victory over Wales in their World Cup qualifier at his former Old Trafford home yesterday.,1,1
4,"Billy Reay, Hawks #39; winningest coach, dead at 86 Billy Reay, the winningest coach in Blackhawks history and a two-time Stanley Cup champion as a player, died Thursday at age 86. Reay, who had suffered from cancer",1,1
5,"Pandas benefit from wireless net The world #39;s dwindling panda population is getting a helping hand from a wireless internet network. The Wolong Nature Reserve in the Sichuan Province of southwest China is home to 20 of the remaining 1,500 giant pandas in the world.",3,3
6,"Stocks Up in Light Volume Ahead of Convention NEW YORK (Reuters) - U.S. stocks gained on Friday after reports showed U.S. economic growth in the second quarter matched economists' forecasts and consumer sentiment picked up above expectations in late August, but volume was feather light ahead of the upcoming Republican convention in New York City.",2,2
7,"Mutual fund votes show limits to change When corporate America came looking for raises last year, Boston #39;s largest mutual funds routinely turned them down, voting in hundreds of cases against lucrative",2,2
8,Rams Offensive Line Strong in Debut (AP) AP - A makeshift offensive line came together at just the right time for the St. Louis Rams.,1,1
9,"Rival Targets Apple's ITunes Customers (AP) AP - For more than a year, Apple Computers Inc. enjoyed singular success selling songs exclusively to users of its iPod portable music player. Now, it's got rival RealNetworks Inc. trying to lure iTunes customers away.",3,3
